In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import unicodedata # Dodajemy unicodedata do normalizacji znaków

def scrape_court_prices(url, csv_filename="cennik_kortow_czyste_ascii.csv"):
    """
    Scrapes court pricing information from the given URL and saves it to a CSV file
    with only ASCII characters and numerical prices (without currency or /h).

    Args:
        url (str): The URL of the webpage to scrape.
        csv_filename (str): The name of the CSV file to save the data to.
                            Defaults to "cennik_kortow_czyste_ascii.csv".

    Returns:
        list: A list of dictionaries, where each dictionary contains
              'days', 'time_range', 'price_regular', and 'price_discounted'
              for a pricing block. Returns an empty list if data cannot be scraped.
    """
    all_court_pricing_data = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        html_content = response.text

        soup = BeautifulSoup(html_content, 'html.parser')

        korty_heading_strong = soup.find('strong', string='Korty:')

        if korty_heading_strong:
            common_parent_of_pricing_blocks = korty_heading_strong.find_parent('div').find_parent('div')

            if common_parent_of_pricing_blocks:
                # Wyszukujemy TEKST Z ORYGINALNYMI POLSKIMI ZNAKAMI, TAK JAK JEST NA STRONIE.
                pricing_blocks = common_parent_of_pricing_blocks.find_all(
                    lambda tag: tag.name == 'div' and tag.find('strong', string=re.compile(r'Poniedziałek – Piątek|Weekendy i święta'))
                )
            else:
                print("Nie znaleziono wspolnego rodzica dla blokow cenowych.")
                return []
        else:
            print("Nie znaleziono naglowka 'Korty:'.")
            return []

        if not pricing_blocks:
            print("Nie znaleziono blokow cenowych. Sprawdz selektory lub strukture strony.")
            return []

        # Funkcja pomocnicza do konwersji tekstu na czyste ASCII i usunięcia niepotrzebnych znaków
        def clean_to_ascii(text):
            if text is None:
                return None
            text = str(text)
            
            # 1. Normalizacja Unicode, by rozłożyć znaki diakrytyczne (np. 'ą' -> 'a' + ogonek)
            # a następnie usunąć te znaki diakrytyczne
            text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
            
            # 2. Zamiana specyficznych znaków i symboli
            text = text.replace(' – ', ' - ').replace('zł/h', '').replace('/h', '').replace('zł', '') # Myślnik, waluta, /h
            text = text.strip() # Usuń spacje na początku i końcu
            
            # 3. Usuń wszystkie niecyfrowe znaki (oprócz kropki dla liczb dziesiętnych)
            # Należy to robić ostrożnie, aby nie usunąć ważnych separatorów
            # Dla cen, usuniemy wszystko, co nie jest cyfrą ani kropką/przecinkiem, a potem wyczyścimy
            
            return text

        # Funkcja do wyciągnięcia samej liczby z ceny
        def extract_price_number(price_text):
            if price_text is None:
                return None
            
            # Usuń wszystkie niecyfrowe znaki oprócz kropek/przecinków
            cleaned_price = re.sub(r'[^\d,.]', '', price_text)
            
            # Jeśli przecinek jest używany jako separator dziesiętny, zamień na kropkę
            if ',' in cleaned_price and '.' not in cleaned_price:
                cleaned_price = cleaned_price.replace(',', '.')
            
            try:
                return float(cleaned_price)
            except ValueError:
                return None


        for block in pricing_blocks:
            data = {}

            days_strong_tag = block.find('strong')
            if days_strong_tag:
                data['days'] = clean_to_ascii(days_strong_tag.get_text(strip=True))

                time_node = days_strong_tag.next_sibling
                if time_node and isinstance(time_node, str):
                    time_raw = time_node.replace('\xa0', ' ').replace('&nbsp;', ' ').strip()
                    time_match = re.search(r'(\d{1,2}:\d{2} – \d{1,2}:\d{2})', time_raw)
                    if time_match:
                        # W zakresie godzin zamień tylko myślnik na zwykły myślnik ASCII
                        data['time_range'] = time_match.group(1).replace(' – ', ' - ')
                    else:
                        data['time_range'] = None
                else:
                    data['time_range'] = None

            all_strong_tags_in_block = block.find_all('strong')

            if len(all_strong_tags_in_block) >= 2:
                price_regular_raw = all_strong_tags_in_block[1].get_text(strip=True)
                data['price_regular'] = extract_price_number(price_regular_raw)

                discount_text_node = all_strong_tags_in_block[1].next_sibling
                if discount_text_node and isinstance(discount_text_node, str):
                    match = re.search(r'\(w karnecie\s*(\d+[,.]?\d*\s*zł/h)\)', discount_text_node) # Regex szuka "zł/h" lub "zl/h"
                    if match:
                        price_discounted_raw = match.group(1)
                        data['price_discounted'] = extract_price_number(price_discounted_raw)
                    else:
                        data['price_discounted'] = None
                else:
                    data['price_discounted'] = None
            else:
                data['price_regular'] = None
                data['price_discounted'] = None

            if data:
                all_court_pricing_data.append(data)

        df = pd.DataFrame(all_court_pricing_data)
        
        # Zapis do pliku CSV z kodowaniem UTF-8.
        # Dane w DataFrame są już oczyszczone do ASCII/liczb.
        # To powinno rozwiązać problem z "krzakami" w programie otwierającym CSV.
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"\nDane zostaly zapisane do pliku '{csv_filename}'")

    except requests.exceptions.RequestException as e:
        print(f"Blad podczas pobierania strony: {e}")
    except Exception as e:
        print(f"Wystapil nieoczekiwany blad: {e}")

    return all_court_pricing_data

# URL strony do zeskrobania
url = "https://www.kortypraga.pl/cennik-2-2/"

# Wywolanie funkcji
pricing_info = scrape_court_prices(url)

if pricing_info:
    print("Zeskrobane dane o cenach kortow (rowniez wyswietlone w konsoli):")
    for item in pricing_info:
        print(item)
else:
    print("Nie udalo sie zeskrobac danych o cenach kortow.")


Dane zostaly zapisane do pliku 'cennik_kortow_czyste_ascii.csv'
Zeskrobane dane o cenach kortow (rowniez wyswietlone w konsoli):
{'days': 'Poniedziaek  Piatek', 'time_range': '7:00 - 17:00', 'price_regular': 65.0, 'price_discounted': 55.0}
{'days': 'Poniedziaek  Piatek', 'time_range': '17:00 - 23:00', 'price_regular': 95.0, 'price_discounted': 85.0}
{'days': 'Weekendy i swieta', 'time_range': '8:00 - 22:00', 'price_regular': 80.0, 'price_discounted': 70.0}
